In [1]:
import numpy as np
from scipy.special import logit,expit
import collections

N=200
P=3
betat1=np.array([1 , 1 , 1])
betat2=np.array([3 , 3 , 3])
betat=np.array([betat1,betat2])

np.random.seed(123)
mu=np.array([-1,-1,-1])
cov=np.array([[2,0.5,0.5],[0.5,2,0.5],[0.5,0.5,2]])
X=np.random.multivariate_normal(mu,cov,N,check_valid='raise')

In [2]:
N=500
P=1

betat1=np.array(1.27745888)
betat2=np.array(-0.07740899)
betat=np.array([betat1,betat2])

np.random.seed(123)
X=np.random.normal(1,1,N)

In [3]:
N=500
P=1

betat3=np.array(0.3)
betat4=np.array(1)
betatt=np.array([betat3,betat4])

np.random.seed(123)
X1=np.random.normal(1,1,N)

In [4]:
def g(x,y):
    y=np.dot(x,y)
    return y

def f(x,y):
    y=np.dot(x,y)
    return y

def pi(x,y):
    y=expit(g(x,y))
    return y

def phi(x,y):
    y=expit(f(x,y))
    return y

def I0(y):
    if y==0:
        return 1

In [5]:
Y=np.zeros(N)
tgroup=np.zeros(N)
pgroup=np.zeros((N,2))
for i in range(N):
    a=np.random.rand()
    b=pi(X[i],betat1)
    pgroup[i,0]=b
    if a>=b:
        tgroup[i]=1
        c=np.random.rand()
        d=phi(X[i],betat2)
        pgroup[i,1]=d
        if c>=d:
            Y[i]=1

In [6]:
Z=np.zeros(N)
Ztgroup=np.zeros(N)
Zpgroup=np.zeros((N,2))
for i in range(N):
    a=np.random.rand()
    b=pi(X[i],betat3)
    pgroup[i,0]=b
    if a>=b:
        Ztgroup[i]=1
        c=np.random.rand()
        d=phi(X[i],betat4)
        Zpgroup[i,1]=d
        if c>=d:
            Z[i]=1

In [7]:
collections.Counter(Z),collections.Counter(Ztgroup)

(Counter({0.0: 421, 1.0: 79}), Counter({1.0: 221, 0.0: 279}))

In [8]:
collections.Counter(Y),collections.Counter(tgroup)

(Counter({0.0: 432, 1.0: 68}), Counter({1.0: 125, 0.0: 375}))

In [9]:
#E-step
def Lih(X,beta):
    Ga=np.zeros((N,2))

    for i in range(N):#响应度
        if Y[i]==1:
            Ga[i,0]=0
            Ga[i,1]=1
        elif Y[i]==0:
            a=(expit(np.dot(X[i],beta[0])))
            b=(1-a)*expit(np.dot(X[i],beta[1]))
            Ga[i,0]=a/(a+b)
            Ga[i,1]=1-Ga[i,0]

    return Ga

In [10]:
#M-step
def Q(X,Y,beta):
    Ga=Lih(X,beta)
    A=np.zeros(N)
    for i in range(N):
        pi=expit(np.dot(beta[0],X[i]))
        phi=expit(np.dot(beta[1],X[i]))
        if Y[i]==0:
            A[i]=(Ga[i,0])*np.log(pi)+(Ga[i,1])*np.log((1-pi)*(1-phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)))
        else:
            A[i]=(Ga[i,1])*np.log((1-pi)*(phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(phi)))
        #A[i]=(Ga[i,0]*(1-Y[i])*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)+Y[i]*np.dot(X[i],beta[1])))
    Q=np.sum(A)
    return Q

def grad(X,Y,Ga,beta):
    A=np.zeros((N,P))
    B=np.zeros((N,P))
    for i in range(N):
        A[i]=((Ga[i,0]*(1-Y[i]))*(1-expit(np.dot(beta[0],X[i])))-(Ga[i,1]*expit(np.dot(beta[0],X[i]))))*X[i]
        B[i]=(Ga[i,1]*(Y[i]-expit(np.dot(beta[1],X[i]))))*X[i]
    grad1=np.sum(A,axis=0)
    grad2=np.sum(B,axis=0)
    grad=np.array([grad1,grad2])
    return grad



In [11]:
def dgra(X,Y,ibeta,n=200,t=0.01,xim=0.6,yim=0.3,zim=0.001):#梯度下降（回溯梯度下降法）
    
    L=Lih(X,ibeta)#E-step
    for i in range(n):#M-step
        lik1=Q(X,Y,ibeta)
        ibeta=ibeta+t*grad(X,Y,L,ibeta)
        lik=Q(X,Y,ibeta)
        d=np.r_[ibeta[0],ibeta[1]]
        c=np.dot(d,d)
        b=(lik-lik1)
        if b>=yim*t*c:
            t=xim*t
        if c<=zim:
            break
    return ibeta


#def dgra1(X,Y,ibeta,n=1000,a=0.01,yim=0.2,zim=0.001):
    
    for i in range(n):
        lik1=Q(X,Y,ibeta)
        L=Lih(X,ibeta)#E-step
        ibeta=ibeta+a*grad(X,Y,L,ibeta)
        lik=Q(X,Y,ibeta)
        d=np.r_[ibeta[0],ibeta[1]]
        c=np.dot(d,d)
        b=(lik-lik1)
        if b>=yim*a*c:
            a=0.5*a
        if c<=zim:
            break
    return ibeta

In [12]:
betat,betatt

(array([ 1.27745888, -0.07740899]), array([0.3, 1. ]))

In [13]:
#initial value
beta01=[-1]
beta02=[5]
beta0=np.array([beta01,beta02])

In [14]:
Q(X,Y,betat),Q(X,Y,betatt)

(-341.19426629636564, -586.8181385034222)

In [15]:
EMbeta=beta0
for i in range(100):
    lik1=Q(X,Y,EMbeta)
    EMbeta=dgra(X,Y,EMbeta)
    lik=Q(X,Y,EMbeta)
    print(i)
    if lik-lik1<=0.0001:
        break
EMbeta,Q(X,Y,EMbeta)

0
1
2
3
4
5
6
7
8


(array([[ 1.15981793],
        [-0.036802  ]]),
 -350.2901472910196)

In [16]:
Ga=Lih(X,betat)
A=np.zeros(N)
for i in range(N):
    pi=expit(np.dot(betat[0],X[i]))
    phi=expit(np.dot(betat[1],X[i]))
    if Y[i]==0:
        A[i]=(Ga[i,0])*np.log(pi)+(Ga[i,1])*np.log((1-pi)*(1-phi))
        #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
        #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)))
    else:
        A[i]=(Ga[i,1])*np.log((1-pi)*(phi))
        #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
        #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(phi)))
    #A[i]=(Ga[i,0]*(1-Y[i])*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)+Y[i]*np.dot(X[i],beta[1])))
Q=np.sum(A)

In [17]:
Q

-341.19426629636564

In [18]:
A

array([-0.95999757, -3.40003799, -0.37038541, -1.09490742, -0.73086648,
       -0.09278317, -1.15123616, -0.66071201, -0.14010443, -0.86530509,
       -0.77816831, -0.51274242, -0.11026224, -0.75932309, -1.82497267,
       -0.66323617, -0.05035518, -0.05144401, -3.40827014, -0.3369298 ,
       -0.24053801, -0.11034   , -0.89625626, -0.15397501, -1.02411674,
       -0.75878032, -0.20297082, -1.13305612, -0.53179905, -0.8630422 ,
       -0.58195422, -0.721717  , -1.15076857, -0.7880402 , -0.198827  ,
       -0.54613942, -2.2588969 , -0.25246524, -0.87109618, -0.37016763,
       -0.83721594, -1.14468532, -1.87304194, -0.2821583 , -0.35206246,
       -0.47893712, -0.04084716, -0.3286545 , -0.18871741, -0.04857279,
       -1.03800455, -0.94064186, -0.0839227 , -0.83384279, -0.46252186,
       -0.17196446, -0.20636358, -0.08290556, -0.10976035, -0.17195086,
       -0.82209096, -0.22719472, -0.35998933, -1.18690974, -0.1193511 ,
       -0.2244073 , -0.45636547, -0.57200692, -1.00395862, -0.39